In [6]:
from tkinter import *
#Creating main window


def Input_Box():
    root = Tk()
    # creating a top window
    master_2 = Toplevel(root)

    #Textboxes
    user_name = Entry(master_2)
    user_name.grid(row = 1, column = 2)
    pwd = Entry(master_2)
    pwd.grid(row = 2, column = 2)

    label_un = Label(master_2, text = "Username")
    label_un.grid(row = 1, column = 1)
    label_pwd = Label(master_2, text = "Password")
    label_pwd.grid(row = 2, column = 1)

    get_button = Button(master_2, text = "Confirm", command = lambda: getname(user_name))
    get_button.grid(row=3, column = 1)3
    master_2.mainloop() 
    
    
    #call_button = Button(root, text='Enter Usrnm and pwd', command = Input_Box)
    #call_button.pack()
    
def getname(user_name):
    input = user_name.get()
    print("!!!!!: %s"%input)




In [7]:
from tkinter import *

class GUI(object):
    def changeLabel(self):
        text = "You have entered " + self.someName.get()
        self.labelText.set(text)
        self.someName.delete(0, END)
        self.someName.insert(0, "You've clicked!")            
    def processed_text(self):
        if  self.someName.get() is None:
            return ""
        else:
            text = re.sub('[\n\r]', ' ',  self.someName.get())
            return text
    def __init__(self):
        app = Tk()
        app.title("GUI Test")
        app.geometry('450x300')

        self.labelText = StringVar()
        self.labelText.set("Click when ready")
        label1 = Label(app, textvariable=self.labelText, height=4)
        label1.pack()

        userInput = StringVar(None)
        self.someName = Entry(app, textvariable=userInput)
        self.someName.pack()

        button1 = Button(app, text="Click Here", width=20,command=self.changeLabel)
        button1.pack(side='bottom',padx=15,pady=15)
        text = self.processed_text
        print
        app.mainloop()

GUI() #calling the class to run

In [25]:
import json
from json import JSONDecodeError
import bs4
import requests
import re
import logging as log
from abc import ABCMeta, abstractmethod
import sys
import numpy as np
from textblob import TextBlob
from snownlp import SnowNLP
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess
from azure.storage.blob import ContentSettings
from math import pow
from easygui import *

tArray = []
iArray = []
class InstagramUser:
    def __init__(self, user_id, username=None, bio=None, followers_count=None, following_count=None, is_private=False):
        """
        A class to represent an Instagram User
        :param user_id: User ID of instagram user
        :param username: Username of Instagram user
        :param bio: Bio text for user
        :param followers_count: Number of followers
        :param following_count: Number of people following
        :param is_private: Boolean to indicate if account is private or not
        """
        self.id = user_id
        self.username = username
        self.bio = bio
        self.followers_count = followers_count
        self.following_count = following_count
        self.is_private = is_private


class InstagramPost:
    def __init__(self, post_id, code, user=None, caption="", display_src=None, is_video=False, created_at=None):
        """
        A class to represent a post on Instagram
        :param post_id: ID of the post
        :param code: Code of the post
        :param user: A user object representing the owner of the post
        :param caption: The caption/text of the post
        :param display_src: The URL of the image of the post
        :param is_video: A boolean value indicating it's a video
        :param created_at: The time it was created
        """
        self.post_id = post_id
        self.code = code
        self.caption = caption
        self.user = user
        self.display_src = display_src
        self.is_video = is_video
        self.created_at = created_at

    def processed_text(self):
        """
        Processes a caption to remove newlines in it.
        :return:
        """
        if self.caption is None:
            return ""
        else:
            text = re.sub('[\n\r]', ' ', self.caption)
            return text
    def processed_image(self):
        """
        Processes a caption to remove newlines in it.
        :return:
        """
        if self.display_src is None:
            return ""
        else:
            image = re.sub('[\n\r]', ' ', self.display_src)
            return image

    def hashtags(self):
        """
        Simple hashtag extractor to return the hastags in the post
        :return:
        """
        hashtags = []
        if self.caption is None:
            return hashtags
        else:
            for tag in re.findall("#[a-zA-Z0-9]+", self.caption):
                hashtags.append(tag)
            return hashtags


class HashTagSearch(metaclass=ABCMeta):
    instagram_root = "https://www.instagram.com"

    def __init__(self, ):
        """
        This class performs a search on Instagrams hashtag search engine, and extracts posts for that given hashtag.
        There are some limitations, as this does not extract all occurrences of the hash tag.
        Instead, it extracts the most recent uses of the tag.
        """
        super().__init__()

    def extract_recent_tag(self, tag):
        """
        Extracts Instagram posts for a given hashtag
        :param tag: Hashtag to extract
        """
        
        url_string = "https://www.instagram.com/%s/"%tag
        print(url_string)
        response = bs4.BeautifulSoup(requests.get(url_string).text, "html.parser")
        ##print("!!!!... Response: %s"%response)
        potential_query_ids = self.get_query_id(response)
        shared_data = self.extract_shared_data(response)
        ##print("@@@....share data :%s"% shared_data)
        media = shared_data['entry_data']['ProfilePage'][0]['user']['media']
        #print("@@@..... media"%media)
        posts = []
        for node in media['nodes']:
            ##print("!!!!.....NODE %s"%node);
            post = self.extract_recent_instagram_post(node)
            posts.append(post)
        self.save_results(posts)

        end_cursor = media['page_info']['end_cursor']

        # figure out valid queryId
        success = False
        for potential_id in potential_query_ids:
            url = "https://www.instagram.com/graphql/query/?query_id=%s&tag_name=%s&first=12&after=%s" % (
                potential_id, tag, end_cursor)
            try:
                data = requests.get(url).json()
                if 'hashtag' not in data['data']:
                    # empty response, skip
                    continue
                query_id = potential_id
                success = True
                break
            except JSONDecodeError as de:
                # no valid JSON retured, most likely wrong query_id resulting in 'Oops, an error occurred.'
                pass
        if not success:
            log.error("Error extracting Query Id, exiting")
            sys.exit(1)

        while end_cursor is not None:
            url = "https://www.instagram.com/graphql/query/?query_id=%s&tag_name=%s&first=12&after=%s" % (
            query_id, tag, end_cursor)
            data = json.loads(requests.get(url).text)
            end_cursor = data['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
            posts = []
            for node in data['data']['hashtag']['edge_hashtag_to_media']['edges']:
                posts.append(self.extract_recent_query_instagram_post(node['node']))
            self.save_results(posts)


    @staticmethod
    def extract_shared_data(doc):
        for script_tag in doc.find_all("script"):
            if script_tag.text.startswith("window._sharedData ="):
                shared_data = re.sub("^window\._sharedData = ", "", script_tag.text)
                shared_data = re.sub(";$", "", shared_data)
                shared_data = json.loads(shared_data)
                return shared_data

    @staticmethod
    def extract_recent_instagram_post(node):
        return InstagramPost(
            post_id=node['id'],
            code=node['code'],
            user=InstagramUser(user_id=node['owner']['id']),
            caption=node['caption'] if 'caption' in node else None,
            display_src=node['display_src'],
            is_video=node['is_video'],
            created_at=node['date']
        )

    @staticmethod
    def extract_recent_query_instagram_post(node):
        return InstagramPost(
            post_id=node['id'],
            code=node['shortcode'],
            user=InstagramUser(user_id=node['owner']['id']),
            caption=node['edge_media_to_caption']['edges'][0]['node']['text']
            if len(node['edge_media_to_caption']['edges']) > 0 else None,
            display_src=node['display_url'],
            is_video=node['is_video'],
            created_at=node['taken_at_timestamp']
        )

    @staticmethod
    def extract_owner_details(owner):
        """
        Extracts the details of a user object.
        :param owner: Instagrams JSON user object
        :return: An Instagram User object
        """
        username = None
        if "username" in owner:
            username = owner["username"]
        is_private = False
        if "is_private" in owner:
            is_private = is_private
        user = InstagramUser(owner['id'], username=username, is_private=is_private)
        return user

    def get_query_id(self, doc):
        query_ids = []
        for script in doc.find_all("script"):
            if script.has_attr("src") and "en_US_Commons" in script['src']:
                text = requests.get("%s%s" % (self.instagram_root, script['src'])).text
                for query_id in re.findall("(?<=queryId:\")[0-9]{17,17}", text):
                    query_ids.append(query_id)
        return query_ids

    @abstractmethod
    def save_results(self, instagram_results):
        """
        Implement yourself to work out what to do with each extract batch of posts
        :param instagram_results: A list of Instagram Posts
        """


class HashTagSearchExample(HashTagSearch):
    def __init__(self):
        super().__init__()
        self.total_posts = 0

    def save_results(self, instagram_results):
        super().save_results(instagram_results)
        #print(instagram_results)
        for i, post in enumerate(instagram_results):
            self.total_posts += 1
            #print("%i - %s" % (self.total_posts, post.processed_text()))
            #print("%s"% (post.processed_image()))
            tArray.append(post.processed_text())
            iArray.append(post.processed_image())
        #print (iArray)
        #print (tArray)
        a = []
        average_sad = []
        average_happy = []
        image_sad = []
        image_happy = []
        j=0
        
        for text in tArray:
            n_text = text.split(" ")
            a = []
            new_text = ""
            count=0
            total=0
            print("len: %i"%len(n_text))  
            for i in range(0,len(n_text)):
                new_text = ""
                for ch in n_text[i]:
                    if ord(ch) > 0x4e00 and ord(ch) < 0x9fff:
                        new_text+=ch
                a.append(new_text)
                if n_text[i] == ''or new_text=='':
                    continue
                else :
                    s =''.join(a[i])
                    s = SnowNLP(s)
                    s1 = SnowNLP(s.sentences[0])
                    count+=s1.sentiments
                    print("Sentence %i  : %s || Score %f"% (len(a) , a[i], s1.sentiments))
                    total = total+1
            #print("total=%i"%total)
            if total==0:
                avv=0.5
            else:
                avv = count/total
            if avv>=0.4:
                average_happy.append(avv)
                image_happy.append(iArray[tArray.index(text)])
            else:
                average_sad.append(avv)
                #print("aaaaa%s"%iArray[tArray.index(text)])
                image_sad.append(iArray[tArray.index(text)])
            #average.append(round(count/len(n_text),2))
            print("Average : %.2f"%avv)
            
            print("---------------------------------------------")
        np.savetxt('image_happy.txt', image_happy, fmt='%s')
        np.savetxt('image_sad.txt', image_sad, fmt='%s')
        np.savetxt('score_happy.txt', average_happy, fmt='%.2f')
        np.savetxt('score_sad.txt', average_sad, fmt='%.2f')
        block_blob_service = BlockBlobService(account_name='storage19960311', account_key='/VwwrK4FVgSFaweKkOv4TTX/2ymEzcnSIhfx6jTRBK9S7xzOIPXMMi4DVT8CytzoP+YkQhdmXUcj7QS1JBtXUg==')
        block_blob_service.create_container('mycontainer', public_access=PublicAccess.Container)
        block_blob_service.create_blob_from_path(
            'mycontainer',
            'imagehappy.txt',
            'image_happy.txt',
            content_settings=ContentSettings(content_type='application/TXT')
            )
        block_blob_service.create_blob_from_path(
            'mycontainer',
            'imagesad.txt',
            'image_sad.txt',
            content_settings=ContentSettings(content_type='application/TXT')
            )
        block_blob_service.create_blob_from_path(
            'mycontainer',
            'scorehappy.txt',
            'score_happy.txt',
            content_settings=ContentSettings(content_type='application/TXT')
            )
        block_blob_service.create_blob_from_path(
            'mycontainer',
            'scoresad.txt',
            'score_sad.txt',
            content_settings=ContentSettings(content_type='application/TXT')
            )


        
if __name__ == '__main__':
    
    log.basicConfig(level=log.INFO)
    title = "Hackthon DEMO"

    #msgbox displays the string you give as an argument and gives the user an OK
    #button.
    #msgbox("Welcome to the demo GUI!", title)

    #You can specify the text of the OK button using the ok_button parameter.
    #msgbox("Are you read for some UI magic?", title, ok_button="I'm ready!")
    band = enterbox("Please Enter Your IG Account?", title)
    #print ("%s"%band)
    HashTagSearchExample().extract_recent_tag(band)


https://www.instagram.com/ping7446/


INFO:azure.storage.common.storageclient:Client-Request-ID=89d869d4-d258-11e7-bb7b-6c4008956dc6 Outgoing request: Method=PUT, Path=/mycontainer, Query={'restype': 'container', 'timeout': None}, Headers={'x-ms-blob-public-access': 'container', 'x-ms-version': '2017-04-17', 'User-Agent': 'Azure-Storage/0.37.1-0.37.1 (Python CPython 3.6.3; Darwin 16.7.0)', 'x-ms-client-request-id': '89d869d4-d258-11e7-bb7b-6c4008956dc6', 'x-ms-date': 'Sun, 26 Nov 2017 03:18:56 GMT', 'Authorization': 'SharedKey storage19960311:BXUQDMEG/FzDxwzxxwLXajfR8v59MEx9XSQHr2FEq/M='}.


len: 3
Sentence 1  : 今天我的工作是幫忙拍攝 || Score 0.720436
Sentence 2  : 他們要提供給報紙刊登 || Score 0.033009
Sentence 3  : 很開心自己也能幫的上忙 || Score 0.665949
Average : 0.47
---------------------------------------------
len: 5
Sentence 1  : 亞歷山大 || Score 0.479501
Sentence 2  : 今天下午是亞歷體能訓練測驗 || Score 0.031049
Sentence 3  : 亞歷爸也是工作人員之幫忙 || Score 0.437414
Sentence 4  : 很棒的體驗 || Score 0.827937
Sentence 5  : 我跟阿弟就負責加油 || Score 0.831982
Average : 0.52
---------------------------------------------
len: 5
Sentence 1  : 父子 || Score 0.909091
Sentence 2  : 認真挑選菜單中 || Score 0.807464
Average : 0.86
---------------------------------------------
len: 3
Sentence 1  : 再忙也要好好吃頓飯 || Score 0.293135
Average : 0.29
---------------------------------------------
len: 3
Sentence 1  : 安德里 || Score 0.500000
Sentence 2  : 你在看我嗎 || Score 0.348244
Sentence 3  : 我也在看你 || Score 0.587521
Average : 0.48
---------------------------------------------
len: 12
Sentence 1  : 安德里 || Score 0.500000
Sentence 2  : 帶阿弟來吃午飯 || Score 0.735570
Sentence

INFO:azure.storage.common.storageclient:Client-Request-ID=89d869d4-d258-11e7-bb7b-6c4008956dc6 Receiving Response: Server-Timestamp=Sun, 26 Nov 2017 03:18:56 GMT, Server-Request-ID=4cb16d36-001e-0086-3f65-66d999000000, HTTP Status Code=409, Message=The specified container already exists., Headers={'x-ms-blob-public-access': 'container', 'x-ms-version': '2017-04-17', 'User-Agent': 'Azure-Storage/0.37.1-0.37.1 (Python CPython 3.6.3; Darwin 16.7.0)', 'x-ms-client-request-id': '89d869d4-d258-11e7-bb7b-6c4008956dc6', 'x-ms-date': 'Sun, 26 Nov 2017 03:18:56 GMT', 'Authorization': 'SharedKey storage19960311:BXUQDMEG/FzDxwzxxwLXajfR8v59MEx9XSQHr2FEq/M='}.
INFO:azure.storage.common.storageclient:Client-Request-ID=89d869d4-d258-11e7-bb7b-6c4008956dc6 Operation failed: checking if the operation should be retried. Current retry count=0, Server-Timestamp=Sun, 26 Nov 2017 03:18:56 GMT, Server-Request-ID=4cb16d36-001e-0086-3f65-66d999000000, HTTP status code=409, Exception=The specified container alr

INFO:azure.storage.common.storageclient:Client-Request-ID=8a141c36-d258-11e7-938a-6c4008956dc6 Receiving Response: Server-Timestamp=Sun, 26 Nov 2017 03:18:57 GMT, Server-Request-ID=4cb16d5e-001e-0086-4d65-66d999000000, HTTP Status Code=201, Message=Created, Headers={'x-ms-blob-type': 'BlockBlob', 'x-ms-lease-id': None, 'If-Modified-Since': None, 'If-Unmodified-Since': None, 'If-Match': None, 'If-None-Match': None, 'x-ms-blob-cache-control': None, 'x-ms-blob-content-type': 'application/TXT', 'x-ms-blob-content-disposition': None, 'x-ms-blob-content-md5': None, 'x-ms-blob-content-encoding': None, 'x-ms-blob-content-language': None, 'Content-Length': '20', 'x-ms-version': '2017-04-17', 'User-Agent': 'Azure-Storage/0.37.1-0.37.1 (Python CPython 3.6.3; Darwin 16.7.0)', 'x-ms-client-request-id': '8a141c36-d258-11e7-938a-6c4008956dc6', 'x-ms-date': 'Sun, 26 Nov 2017 03:18:57 GMT', 'Authorization': 'SharedKey storage19960311:/KM8s6oe0Z/siOspwtqt1wy1cTd5xOSX4CwJ0UmPUrk='}.
ERROR:root:Error extr

SystemExit: 1

/Users/helen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from easygui import *
title = "GUI DEMO"

#msgbox displays the string you give as an argument and gives the user an OK
#button.
msgbox("Welcome to the demo GUI!", title)

#You can specify the text of the OK button using the ok_button parameter.
msgbox("Are you read for some UI magic?", title, ok_button="I'm ready!")
band = enterbox("What's your name?", title)
print ("%s"%band)